In [ ]:
# SIT796 - Task 1.2 Policy Definition
# Set up OpenAI gym environment

# install required system dependencies
# install required system dependencies
!apt-get install -y xvfb x11-utils 
!apt-get install x11-utils > /dev/null 2>&1
!pip install PyVirtualDisplay==2.0.* \
  PyOpenGL==3.1.* \
  PyOpenGL-accelerate==3.1.* \
  gym[box2d]==0.17.* 
!pip install pyglet

# import other libraries
import gym
import numpy as np
import base64
import io
import IPython


Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
# Custom House Cleaning Environment Class
from gym.wrappers import Monitor
from IPython import display
from pyvirtualdisplay import Display
from gym import spaces

class CleanEnv(gym.Env):

  # constructor
  def __init__(self):
    
    # Action Space: the number of actions the agent can take
    
    # The agent can move in 4 directions - north, south, east and west
    # additionally we consider the current state of the tile
    # and lastly we consider if the entire house has been cleaned
    # 9 actions total
    self.action_space = spaces.Discrete(9)

    # Observation Space:
    # we use an array to represent our house with tiles structure
    # x coordinate is the row starting from 0 to 2 (3 tiles down)
    # y coordinate is the column starting from 0 to 8 (9 tiles across)
    # top left is 0,0 and bottom right in our scenario is 2,8

    # the value of each element in the house array represents tile state:
    # 0 = not clean
    # 1 = clean
    #self.observation_space = spaces.Box(low=0, high=1, shape=(3, 9), dtype=np.int32)
    self.observation_space = spaces.Box(np.zeros((3,9)),high=1)
    self.state = np.zeros((3,9))
  
  # step function
  def step(self, action):
    #1. Update the environment state based on the action chosen
    #2. Calculate the "reward" for the new state
    #3. Store the new "observation" for the state
    #4. Check if the episode is over and store as "done"

    # if we cleaned a tile then give a substantial reward
    if action>=0 and action<=3:
      reward = 10
      done = False
    # ***One change to environment between task 1.1 and 1.2
    # if we completed cleaning the entire house give more substantial reward
    elif action==8:
      reward = 100
      done = True
    # if we did not clean give a minor negative reward
    else:
      reward = -1
      done = False

    info = {}
    # Execute one time step within the environment
    return self.state, reward, done, info

  # reset function
  def reset(self):
    # Reset the state of the environment to an initial state
    self.state = np.zeros((3,9))
    return self.state
  
  # no render for this task


In [ ]:
# Policy function with 2 inputs current coordinates (obs) and house array

def policy(obs, house):

    # obs will be the agent's current position represented by an array

    # Constraint: agent cannot move through walls
    # walls are between:
    # 0,2 and 0,3
    # 2,2 and 2,3
    # 0,5 and 0,6
    # 2,5 and 2,6
    # therefore movement between these 4 elements is forbidden
    # additionally agent cannot move outside of the house structure
    # ie. (x < 0 or x > 2) and (y < 0 or y > 8)

    # consider action values:
    # 0 = move north and clean the destination tile
    # 1 = move south and clean the destination tile
    # 2 = move east and clean the destination tile
    # 3 = move west and clean the destination tile
    # 4 = move north and do not clean the destination tile as already clean
    # 5 = move south and do not clean the destination tile as already clean
    # 6 = move east and do not clean the destination tile as already clean
    # 7 = move west and do not clean the destination tile as already clean
    # 8 = all tiles of the house are clean and the task is complete
    
    # there are four possible movements the agent can take
    # north south east and west however we need consider wall restraint
    # lastly the 9th action is when the entire house is clean so task is finished
    
    # a variable for how many tiles have been cleaned
    clean_tiles = np.sum(house)
    # variable for total number of tiles in the environment
    total_tiles = house.size

    print("Possible agent actions:")
    # if all tiles clean then task has been completed
    if clean_tiles == total_tiles:
      action = 8
    else:
      # ideally we want actions that are from 0-3 as that means agent cleans
      # and gets better reward. Initialise an array with value of 9 which is
      # outside of our discrete actions. We will use this to choose the best
      # action
      optimal_value = np.array([9])

      # north and south movements are only barred by the perimeter house wall
      # NORTH
      x = obs[0]
      x = x-1
      y = obs[1]
      # perimeter wall restraint
      if x>=0 and x<=2:
        if y>=0 and y<=8:
          # valid action
          # check destination tile state and store potential action
          if house[x,y] == 0:
            z = "unclean"
            optimal_value = np.append(optimal_value, 0)
          else:
            z = "clean"
            optimal_value = np.append(optimal_value, 4)
          print(" N:",x,",",y,z)
      else:
        print(" N: perimeter wall restraint")
      # SOUTH
      x = obs[0]+1
      y = obs[1]
      # perimeter wall restraint
      if x>=0 and x<=2:
        if y>=0 and y<=8:
          # valid action
          # check destination tile state and store potential action
          if house[x,y] == 0:
            z = "unclean"
            optimal_value = np.append(optimal_value, 1)
          else:
            z = "clean"
            optimal_value = np.append(optimal_value, 5)
          print(" S",x,",",y,z)
      else:
        print(" S: perimeter wall restraint")

      # since the internal walls bar access only from east to west or vice versa
      # EAST
      x = obs[0]
      y = obs[1]+1
      # perimeter wall restraint
      if (x>=0 and x<=2)and(y>=0 and y<=8):
          # internal wall restraint
          if (obs[0] == 0 or obs[0] == 2)and(obs[1] == 2 or obs[1] == 5):
              print(" E: internal wall restraint")
          else:
            # valid action
            # check destination tile state and store potential action
            if house[x,y] == 0:
              z = "unclean"
              optimal_value = np.append(optimal_value, 2)
            else:
              z = "clean"
              optimal_value = np.append(optimal_value, 6)
            print(" E:",x,",",y,z)
      else:
        print(" E: perimeter wall restraint")

      # WEST
      x = obs[0]
      y = obs[1]-1
      # perimeter wall restraint
      if (x>=0 and x<=2)and(y>=0 and y<=8):
          # internal wall restraint
          if (obs[0] == 0 or obs[0] == 2)and(obs[1] == 3 or obs[1] == 6):
              print(" W: internal wall restraint")
          else:
            # valid action
            # check destination tile state and store potential action
            if house[x,y] == 0:
              z = "unclean"
              optimal_value = np.append(optimal_value, 3)
            else:
              z = "clean"
              optimal_value = np.append(optimal_value, 7)
            print(" W:",x,",",y,z)
      else:
        print(" W: perimeter wall restraint")
      
      # remove the value of 9 from the array of actions
      optimal_value = np.delete(optimal_value, np.where(optimal_value == 9))
      # our optimal_value array now has available episode actions

      # DETERMINATION OF NEXT ACTION
      # 1. CLEAN ADJACENT TILES
      # if any are cleaning a tile (actions 0-3), execute that action
      if optimal_value.min()<=3:
        action = optimal_value.min()

      # 2. LINE OF SIGHT
      # assume agent has line of sight of 3 tiles
      # if all tiles directly connected to agent position are clean then
      # we use a line of sight of 3 tiles to determine our best course of action
      # there should be a maximum of 4 actions stored in the array
      # and of these they should be actions 4,5,6 and 7
      # if there are no unclean tiles within 3 tiles of then random action
      # NOTE: the agent has no line of sight through walls. If agent is 
      # next to a wall they cannot see in that direction, the action will not
      # be available in the optimal_value array

      else:
        do_random = True # if this is not changed to False, randomise action
        for i in optimal_value:
          if i == 4: # N
            # check the house array for values of 0 to the north
            # in a 3x3 area, we already cannot move north if at row 0
              x = obs[0] # the row should be 1 or 2
              y = obs[1]
              if y == 0: # limit the line of sight if through past perimeter
                z = 1
              else:
                z = y
              # the los will give us the line of sight 3x3 to the north
              los = house[0:x,z-1:y+2]
              if 0 in los:
                  action = 4 # move north
                  do_random = False
                  print(" LOS: there are tiles to clean to the north")
          elif i == 5: # S
            # check the house array for values of 0 to the south
            # in a 3x3 area, we already cannot move south if at row 2
              x = obs[0] # the row should be 0 or 1
              y = obs[1]
              if y == 0: # limit the line of sight if through past perimeter
                z = 1
              else:
                z = y
              # the los will give us the line of sight 3x3 to the south
              los = house[x+1:3,z-1:y+2]
              if 0 in los:
                  action = 5 # move south
                  do_random = False
                  print(" LOS: there are tiles to clean to the south")
          elif i == 6: # E
            # check the house array for values of 0 to the east
            # in a 3x3 area, we already cannot move east if at row 2
              x = obs[0] # the row should be 0 or 1
              y = obs[1]
              if x == 0: # limit the line of sight if through past perimeter
                z = 1
              else:
                z = x
              # the los will give us the line of sight 3x3 to the east
              los = house[z-1:x+2,y+1:y+4]
              if 0 in los:
                  action = 6 # move east
                  do_random = False
                  print(" LOS: there are tiles to clean to the east")
          elif i == 7: # W
            # check the house array for values of 0 to the west
            # in a 3x3 area, we already cannot move west if at row 2
              x = obs[0] # the row should be 0 or 1
              y = obs[1]
              if x == 0: # limit the line of sight if through past perimeter
                z = 1
              else:
                z = x
              if y<=2: 
                w = 3
              else:
                w = y
              # the los will give us the line of sight 3x3 to the west
              los = house[z-1:x+2,w-3:y]
              if 0 in los:
                  action = 7 # move west
                  do_random = False
                  print(" LOS: there are tiles to clean to the west")
                            
          # 3. NO SUITABLE INFORMATION - TAKE RANDOM ACTION
          if do_random: # if line of sight yields nothing, randomise
            np.random.shuffle(optimal_value)
            action = optimal_value[0]
    return action

In [ ]:
# MAIN FUNCTION

# Set step/time/episode limit
step_limit = 300

# declare environment
env = CleanEnv()

# reset environment to clean slate
o = env.reset()

# In this case our agent will always start at grid tile position 2,3
coord = ([2,3])

# clean the tile that the agent starts on
o[coord[0],coord[1]] = 1

# variable to track total reward
reward = 0

# I will be tracking the agent position in this main function
for t in range(step_limit):
    print("\nStep",t+1," current agent location: ", coord)
    action = policy(coord,o)  # Call policy
    o, r, d, _ = env.step(action)  # Pass the action chosen by the policy to the environment
    
    # move north and clean
    if action==0:
      coord[0] = coord[0]-1
      o[coord[0],coord[1]] = 1
      print(" Action taken: move north and clean the destination tile")
    # move south and clean
    elif action==1:
      coord[0] = coord[0]+1
      o[coord[0],coord[1]] = 1
      print(" Action taken: move south and clean the destination tile")
    # move east and clean
    elif action==2:
      coord[1] = coord[1]+1
      o[coord[0],coord[1]] = 1
      print(" Action taken: move east and clean the destination tile")
    # move west and clean
    elif action==3:
      coord[1] = coord[1]-1
      o[coord[0],coord[1]] = 1
      print(" Action taken: move west and clean the destination tile")
    # move north
    elif action==4:
      coord[0] = coord[0]-1
      print(" Action taken: move north and do not clean the destination tile as already clean")
    # move south
    elif action==5:
      coord[0] = coord[0]+1
      print(" Action taken: move south and do not clean the destination tile as already clean")
    # move east
    elif action==6:
      coord[1] = coord[1]+1
      print(" Action taken: move east and do not clean the destination tile as already clean")
    # move west
    elif action==7:
      coord[1] = coord[1]-1
      print(" Action taken: move west and do not clean the destination tile as already clean")

    # reward tracker
    reward = reward + r
    print(" Current reward:",reward)
    if d and t<step_limit-1:
        print("Task completed in", t, "time/episodes/steps with total reward", reward)
        break
else:
    print("Step limit exceeded. Try again.")


Step 1  current agent location:  [2, 3]
Possible agent actions:
 N: 1 , 3 unclean
 S: perimeter wall restraint
 E: 2 , 4 unclean
 W: internal wall restraint
 Action taken: move north and clean the destination tile
 Current reward: 10

Step 2  current agent location:  [1, 3]
Possible agent actions:
 N: 0 , 3 unclean
 S 2 , 3 clean
 E: 1 , 4 unclean
 W: 1 , 2 unclean
 Action taken: move north and clean the destination tile
 Current reward: 20

Step 3  current agent location:  [0, 3]
Possible agent actions:
 N: perimeter wall restraint
 S 1 , 3 clean
 E: 0 , 4 unclean
 W: internal wall restraint
 Action taken: move east and clean the destination tile
 Current reward: 30

Step 4  current agent location:  [0, 4]
Possible agent actions:
 N: perimeter wall restraint
 S 1 , 4 unclean
 E: 0 , 5 unclean
 W: 0 , 3 clean
 Action taken: move south and clean the destination tile
 Current reward: 40

Step 5  current agent location:  [1, 4]
Possible agent actions:
 N: 0 , 4 clean
 S 2 , 4 unclean
 E: